In [2]:
import numpy as np
import os
from matplotlib import pyplot as plt
from filters.filters import IEKF
from tools.objects import Gaussian
from tools import compare, data_processing, transform, utils
from tools.tools_ahrs import plot
import mrob

In [3]:
x3_path = os.path.join("../data", "TUM-VI", "imu0")
mocap_path = os.path.join("../data", "TUM-VI", "mocap0")
take_name = "outdoors1.csv"

In [4]:
t_base, data_gyr, data_acc = data_processing.import_tum_imu(os.path.join(x3_path, take_name), smoothing=False)

In [5]:
downscale = 1
t_base, data_gyr, data_acc = data_processing.downsample(downscale, t_base, data_gyr, data_acc)

In [6]:
freq_my = 120/downscale # Hz
# if frequency of MoCap is lower than resulting from data - change to MoCap's 240 Hz

In [7]:
data_mocap_t, data_mocap_q, data_mocap_T = data_processing.import_tum_mocap(os.path.join(mocap_path, take_name), smoothing=False)

In [8]:
data_mocap_t, data_mocap_q, data_mocap_T = data_processing.downsample(downscale, data_mocap_t, data_mocap_q, data_mocap_T)

In [9]:
t_base, data_gyr, data_acc, data_acc, data_mocap_q = data_processing.sync_mocap_and_data(data_mocap_t, data_mocap_q, t_base, data_gyr, data_acc, data_acc)

In [ ]:
plot(data_gyr)

In [ ]:
data_mocap_angvel = transform.quats_to_angvels(data_mocap_t, data_mocap_q)
action_cov = utils.calc_cov(data_gyr[:-1], data_mocap_angvel)
print(action_cov)

In [ ]:
g = np.array([0, 0, -9.81])
data_mocap_acc = np.array([np.linalg.inv(mrob.quat_to_so3(data_mocap_q[i])) @ g for i in range(len(data_mocap_q))])
start = 0000
end = 2000
measurement_cov = utils.calc_cov(data_acc[start:end], data_mocap_acc[start:end])
plot(data_acc, data_mocap_acc, data_acc - data_mocap_acc)
print(measurement_cov)

# Not so good results with these covs

In [ ]:

x3_path_my = os.path.join("data", "X3_simple", "Random_walk_2025-03-06_19-02-36.207_TGW")
# x3_path = "madgwick_filter/recordings/X3_simple/Standing_still_2025-03-06_18-50-19.416_TGW"
# x3_path = "madgwick_filter/recordings/X3_simple/Basic_motions_2025-03-06_18-45-09.848_TGW"
# x3_path = "madgwick_filter/recordings/X3_simple/Walking_2025-03-06_18-37-32.144_TGW"
# x3_path = "madgwick_filter/recordings/X3_simple/Walking_talking_2025-03-06_18-41-53.764_TGW"
# x3_path = "madgwick_filter/recordings/X3_simple/Random_walk_2025-03-06_19-02-36.207_TGW"

mocap_path_my = os.path.join("data", "Mocap_simple", "Random_walk_Take 2025-03-06 06.38.58 PM_004.csv")
# mocap_path = "madgwick_filter/recordings/Mocap_simple/Standing_still_Take 2025-03-06 06.38.58 PM_003.csv"
# mocap_path = "madgwick_filter/recordings/Mocap_simple/Basic_motions_Take 2025-03-06 06.38.58 PM_002.csv"
# mocap_path = "madgwick_filter/recordings/Mocap_simple/Walking_Take 2025-03-06 06.38.58 PM.csv"
# mocap_path = "madgwick_filter/recordings/Mocap_simple/Walking_talking_Take 2025-03-06 06.38.58 PM_001.csv"
# mocap_path = "madgwick_filter/recordings/Mocap_simple/Random_walk_Take 2025-03-06 06.38.58 PM_004.csv"


In [ ]:
import_myrecording = False
if import_myrecording:
    t_base, data_gyr, data_acc, data_magn = data_processing.import_combined_data(os.path.join(x3_path_my, "combined_imu_data_7.csv"))
    t_base = t_base/1000
    data_mocap_t, data_mocap_q = data_processing.import_gamerotvec_data(os.path.join(x3_path_my, "game_rotation_vector_12.csv"))
    data_mocap_t, data_mocap_q, t_base, data_gyr, data_acc, data_acc= data_processing.trim_to_min_length(data_mocap_t, data_mocap_q, t_base, data_gyr, data_acc, data_acc)

In [ ]:
plot(data_mocap_q)

In [ ]:
plot(data_mocap_t - data_mocap_t[0])

In [ ]:
mean_prior = np.array([0, 0, 0.])
Sigma_prior = 1e1 * np.eye(3, 3)
M = 1e-4 * np.eye(3, 3)                                 # rad/s         ~(0.00122 ** 2) from sensors.csv
Q = 1e-3 * np.eye(3, 3) * (9.81 ** 2)                   # rads * g^2    ~(0.00239 ** 2) from sensors.csv
# worse results with covs from sensors or from np.cov
#M = np.diag(np.diag(action_cov))
#Q = np.diag(np.diag(measurement_cov))
initial_state = Gaussian(mean_prior, Sigma_prior)
g = np.array([0, 0, -9.81])
u_bias = np.array([0, 0, 0.000])
iekf = IEKF(initial_state, M, Q, g, u_bias)
u = data_gyr
N = len(data_gyr)
dts = np.empty(N)
freq_sm = 200
dts[0] = 1/freq_sm
dts[1:] = np.array([t_base[i+1] - t_base[i] for i in range(N-1)])
preds = np.empty((N, 3))
s_preds = np.empty((N, 3, 3))
ups = np.empty((N, 3))
s_ups = np.empty((N, 3, 3))
Ks = np.empty((N, 3, 3))
IVs = np.empty((N, 3))
gt = transform.quats_to_rpy(data_mocap_q)

In [ ]:
for i in range(N):
    iekf.predict(u[i], dts[i])
    Ks[i], IVs[i] = iekf.update(data_acc[i])
    #iekf.update_fake()
    preds[i, :] = iekf.mu_bar
    s_preds[i] = iekf.Sigma_bar
    ups[i] = iekf.mu
    s_ups[i] = iekf.Sigma

In [ ]:
print(iekf.Sigma_bar)

In [ ]:
compare.plot_covs(ups, s_ups, 0, gt)
compare.plot_covs(ups, s_ups, 1, gt)
compare.plot_covs(ups, s_ups, 2, gt)
plot(ups - gt)

In [ ]:
q1 = transform.rpy_to_quats(gt)
q2 = transform.rpy_to_quats(ups)
plot(q1 - q2)

In [ ]:
compare.errors_estimation(q1, q2)

In [ ]:
compare.errors_estimation_rpy(gt, ups)